## Volve daily production — Data QC & EDA

**Purpose**

This notebook is a scientific, reproducible QC + EDA pass over the processed Volve dataset (`data/processed/volve_daily.csv`).

We focus on:
- Data integrity (missingness, duplicates, time coverage)
- Uptime (`on_stream_hrs`) and shut-in behavior
- Rate/volume signals (`q_oil`, `q_gas`, `q_water`) and uptime-corrected effective rates (`q_*_eff`)
- Producing clean, exportable figures under `reports/figures/`

**Why this matters for the Equinor Reservoir Engineer role**

Reservoir forecasting quality depends on trustworthy time series. This notebook demonstrates how we:
- validate operational data
- quantify uncertainty sources (missing data, regime changes)
- prepare features and targets for forecasting (DCA / data-driven models)



In [1]:
from __future__ import annotations

from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


def find_project_root(start: Path | None = None) -> Path:
    """Find repo root robustly regardless of where the notebook is executed from."""
    p = (start or Path.cwd()).resolve()
    markers = {"Equinor_Job.md", "requirements.txt", "src"}
    for cand in [p] + list(p.parents):
        hits = 0
        for m in markers:
            if (cand / m).exists():
                hits += 1
        if hits >= 2:
            return cand
    # Fallback to current working directory
    return p


PROJECT_ROOT = find_project_root()
DATA_PATH = PROJECT_ROOT / "data" / "processed" / "volve_daily.csv"
FIG_DIR = PROJECT_ROOT / "reports" / "figures"
FIG_DIR.mkdir(parents=True, exist_ok=True)

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 200)

print("Project root:", PROJECT_ROOT)
print("Data path:", DATA_PATH)
print("Figures dir:", FIG_DIR)

if not DATA_PATH.exists():
    raise FileNotFoundError(f"Expected dataset not found: {DATA_PATH}")

df = pd.read_csv(DATA_PATH)
df["date"] = pd.to_datetime(df["date"], errors="coerce")

# Sort + basic checks
_df = df.sort_values(["well", "date"]).reset_index(drop=True)
print("rows:", len(_df), "wells:", _df["well"].nunique())
print("date range:", _df["date"].min(), "..", _df["date"].max())

# New (operational) columns added to preprocessing for reservoir-engineering context
op_cols = [
    "avg_whp",
    "avg_downhole_pressure",
    "avg_choke_size_p",
    "dp_choke_size",
    "bore_wi_vol",
    "flow_kind",
    "well_type",
]

present = [c for c in op_cols if c in _df.columns]
print("Operational columns present:", present)

if present:
    miss = _df[present].isna().mean().sort_values(ascending=False)
    print("\nMissingness fraction (operational columns):")
    print(miss.to_string())

_df.head()


Project root: /Users/taief/Desktop/Norway Dam/equinor
Data path: /Users/taief/Desktop/Norway Dam/equinor/data/processed/volve_daily.csv
Figures dir: /Users/taief/Desktop/Norway Dam/equinor/reports/figures
rows: 15634 wells: 7
date range: 2007-09-01 00:00:00 .. 2016-12-01 00:00:00
Operational columns present: ['avg_whp', 'avg_downhole_pressure', 'avg_choke_size_p', 'dp_choke_size', 'bore_wi_vol', 'flow_kind', 'well_type']

Missingness fraction (operational columns):
avg_choke_size_p         0.429513
avg_downhole_pressure    0.425611
avg_whp                  0.414417
dp_choke_size            0.018805
bore_wi_vol              0.000000
flow_kind                0.000000
well_type                0.000000


,well,date,q_oil,q_gas,q_water,on_stream_hrs,q_oil_eff,q_gas_eff,q_water_eff,avg_downhole_pressure,avg_whp,avg_choke_size_p,dp_choke_size,bore_wi_vol,flow_kind,well_type
0,NO 15/9-F-1 C,2014-04-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.000000,0.00000,0.0,production,WI
1,NO 15/9-F-1 C,2014-04-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.00000,1.003059,0.00000,0.0,production,OP
2,NO 15/9-F-1 C,2014-04-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.00000,0.979008,0.00000,0.0,production,OP
3,NO 15/9-F-1 C,2014-04-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.00000,0.545759,0.00000,0.0,production,OP
4,NO 15/9-F-1 C,2014-04-11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,310.37614,33.09788,1.215987,33.07195,0.0,production,OP


In [2]:
# QC 1: missingness + duplicates

required_cols = [
    "well",
    "date",
    "q_oil",
    "q_gas",
    "q_water",
    "on_stream_hrs",
    "q_oil_eff",
    "q_gas_eff",
    "q_water_eff",
]
missing_cols = [c for c in required_cols if c not in _df.columns]
if missing_cols:
    raise ValueError(f"Missing expected columns: {missing_cols}")

# Missingness
miss = _df[required_cols].isna().mean().sort_values(ascending=False)
miss.to_frame("missing_fraction")



,missing_fraction
on_stream_hrs,0.018229
well,0.000000
date,0.000000
q_oil,0.000000
q_gas,0.000000
q_water,0.000000
q_oil_eff,0.000000
q_gas_eff,0.000000
q_water_eff,0.000000


In [3]:
# Duplicates on (well, date)
dup_n = int(_df.duplicated(subset=["well", "date"]).sum())
print("Duplicate (well,date) rows:", dup_n)

# Shut-in days: define as on_stream_hrs == 0 OR all three fluids are 0
shutin = (_df["on_stream_hrs"].fillna(0) <= 0) | (
    (_df["q_oil"].fillna(0) == 0) & (_df["q_gas"].fillna(0) == 0) & (_df["q_water"].fillna(0) == 0)
)

shutin_rate = shutin.mean()
print(f"Shut-in fraction (rows): {shutin_rate:.3f}")

# Quick summary per well
per_well = (
    _df.assign(is_shutin=shutin)
    .groupby("well")
    .agg(
        n_days=("date", "count"),
        start=("date", "min"),
        end=("date", "max"),
        shutin_frac=("is_shutin", "mean"),
        mean_uptime=("on_stream_hrs", "mean"),
        max_oil=("q_oil", "max"),
    )
    .sort_values("n_days", ascending=False)
)
per_well


Duplicate (well,date) rows: 0
Shut-in fraction (rows): 0.488


,n_days,start,end,shutin_frac,mean_uptime,max_oil
well,,,,,,
NO 15/9-F-4 AH,3327,2007-09-01,2016-12-01,1.000000,20.240951,0.00
NO 15/9-F-5 AH,3306,2007-09-01,2016-09-18,0.960980,19.170095,396.80
NO 15/9-F-12 H,3056,2008-02-12,2016-09-17,0.071662,21.335109,5901.84
NO 15/9-F-14 H,3056,2008-02-12,2016-09-17,0.108966,20.539878,5644.37
NO 15/9-F-11 H,1165,2013-07-08,2016-09-17,0.036910,22.321809,2064.61
NO 15/9-F-15 D,978,2014-01-12,2016-09-17,0.215746,18.225800,513.12
NO 15/9-F-1 C,746,2014-04-07,2016-04-21,0.423592,13.381412,1549.81


In [4]:
# Visualization 1: oil rate time series per well (daily-average vs effective flowing)

import re


def safe_filename(s: str) -> str:
    # Replace anything that can break paths (/, spaces, etc.)
    return re.sub(r"[^A-Za-z0-9._-]+", "_", s).strip("_")


def plot_well(well: str, col: str, title: str, fname: str) -> None:
    d = _df[_df["well"] == well].copy()
    d = d.sort_values("date")

    fig, ax = plt.subplots(figsize=(12, 4))
    ax.plot(d["date"], d[col], lw=1.2)
    ax.set_title(title)
    ax.set_xlabel("Date")
    ax.set_ylabel(col)
    ax.grid(True, alpha=0.3)

    out = FIG_DIR / fname
    fig.tight_layout()
    fig.savefig(out, dpi=160)
    plt.close(fig)


wells = per_well.index.tolist()
print("Wells:", wells)

# Export two figures per well (oil avg and oil effective)
for w in wells:
    wname = safe_filename(w)
    plot_well(w, "q_oil", f"{w} — Daily-average oil volume (q_oil)", f"{wname}_q_oil.png")
    plot_well(w, "q_oil_eff", f"{w} — Effective flowing oil rate (q_oil_eff)", f"{wname}_q_oil_eff.png")

print("Saved figures to:", FIG_DIR)



Wells: ['NO 15/9-F-4 AH', 'NO 15/9-F-5 AH', 'NO 15/9-F-12 H', 'NO 15/9-F-14 H', 'NO 15/9-F-11 H', 'NO 15/9-F-15 D', 'NO 15/9-F-1 C']


Saved figures to: /Users/taief/Desktop/Norway Dam/equinor/reports/figures
